# Clothes Classification

In [ ]:
import pandas as pd
import tensorflow as tf
from tensorflow import keras

In [ ]:
tf.__version__

In [ ]:
DATA_DIR = "../input/clothing-dataset-full/images_original/"
DATA_RAW = "../input/clothing-dataset-full/images.csv"
MODEL_PATH = "../model.h5"
WIDTH = 150
HEIGHT = 150
NUM_CHANNELS = 3
VALIDATION_SPLIT = 0.2
BATCH_SIZE = 192
LEARNING_RATE = 1e-4
EPOCHS = 5

## Prepare Data

In [ ]:
data = pd.read_csv(DATA_RAW)
data = data.sample(len(data))
data.head(10)

In [ ]:
data["image_path"] = DATA_DIR + data["image"] + ".jpg"
data.head()

In [ ]:
generator = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255,
    validation_split=VALIDATION_SPLIT
)
train_set = generator.flow_from_dataframe(
    dataframe=data,
    x_col="image_path",
    y_col="label",
    target_size=(WIDTH, HEIGHT),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='training',
    shuffle=True
)
val_set = generator.flow_from_dataframe(
    dataframe=data,
    x_col="image_path",
    y_col="label",
    target_size=(WIDTH, HEIGHT),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='validation',
    shuffle=False
)
num_classes = len(train_set.class_indices)
print("Total classes:", num_classes)

## Design model

In [ ]:
model = keras.Sequential([
    keras.Input(shape=(WIDTH, HEIGHT, NUM_CHANNELS)),
    keras.layers.Conv2D(128, (5,5), activation=tf.nn.relu),
    keras.layers.Conv2D(128, (5,5), activation=tf.nn.relu),
    keras.layers.MaxPooling2D(2, 2),
    keras.layers.Conv2D(64, (5,5), activation=tf.nn.relu),
    keras.layers.Conv2D(64, (5,5), activation=tf.nn.relu),
    keras.layers.MaxPooling2D(2, 2),
    keras.layers.Conv2D(32, (5,5), activation=tf.nn.relu),
    keras.layers.Conv2D(32, (5,5), activation=tf.nn.relu),
    keras.layers.MaxPooling2D(2, 2),
    keras.layers.Flatten(),
    keras.layers.Dense(128, activation=tf.nn.relu),
    keras.layers.Dense(num_classes, activation=tf.nn.softmax)
])
model.summary()

## Train model

In [ ]:
model.compile(loss="categorical_crossentropy",
              optimizer=keras.optimizers.RMSprop(lr=LEARNING_RATE),
              metrics=["accuracy"])

_ = model.fit(train_set, epochs=EPOCHS)

## Evaluate model

In [ ]:
_ = model.evaluate(val_set)

## Save model

In [ ]:
model.save(MODEL_PATH)